## צ'אטבוט אינטראקטיבי Phi 3 Mini 4K עם Whisper

### הקדמה:
צ'אטבוט אינטראקטיבי Phi 3 Mini 4K הוא כלי שמאפשר למשתמשים לתקשר עם הדמו של Microsoft Phi 3 Mini 4K באמצעות קלט טקסט או אודיו. ניתן להשתמש בצ'אטבוט למגוון משימות, כמו תרגום, עדכוני מזג אוויר ואיסוף מידע כללי.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


צור אסימון גישה חדש  
ספק שם חדש  
בחר הרשאות כתיבה  
העתק את האסימון ושמור אותו במקום בטוח


הקוד הבא כתוב בפייתון ומבצע שתי משימות עיקריות: ייבוא המודול `os` והגדרת משתנה סביבה.

1. ייבוא המודול `os`:
   - המודול `os` בפייתון מספק דרך לתקשר עם מערכת ההפעלה. הוא מאפשר לבצע משימות שונות הקשורות למערכת ההפעלה, כמו גישה למשתני סביבה, עבודה עם קבצים ותיקיות, ועוד.
   - בקוד הזה, המודול `os` מיובא באמצעות הפקודה `import`. פקודה זו מאפשרת להשתמש בפונקציונליות של המודול `os` בתוך הסקריפט הנוכחי בפייתון.

2. הגדרת משתנה סביבה:
   - משתנה סביבה הוא ערך שניתן לגשת אליו על ידי תוכניות שרצות על מערכת ההפעלה. זהו אמצעי לאחסון הגדרות תצורה או מידע אחר שניתן להשתמש בו על ידי תוכניות שונות.
   - בקוד הזה, משתנה סביבה חדש מוגדר באמצעות המילון `os.environ`. המפתח של המילון הוא `'HF_TOKEN'`, והערך מוקצה מתוך המשתנה `HUGGINGFACE_TOKEN`.
   - המשתנה `HUGGINGFACE_TOKEN` מוגדר ממש מעל קטע הקוד הזה, והוא מקבל ערך מחרוזת `"hf_**************"` באמצעות התחביר `#@param`. תחביר זה משמש לעיתים קרובות במחברות Jupyter כדי לאפשר קלט משתמש והגדרת פרמטרים ישירות בממשק המחברת.
   - על ידי הגדרת משתנה הסביבה `'HF_TOKEN'`, ניתן לגשת אליו על ידי חלקים אחרים בתוכנית או על ידי תוכניות אחרות שרצות על אותה מערכת הפעלה.

בסך הכל, הקוד הזה מייבא את המודול `os` ומגדיר משתנה סביבה בשם `'HF_TOKEN'` עם הערך שסופק במשתנה `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

קטע הקוד הזה מגדיר פונקציה בשם clear_output שמשמשת לניקוי הפלט של התא הנוכחי ב-Jupyter Notebook או IPython. בואו נפרק את הקוד ונבין את הפונקציונליות שלו:

הפונקציה clear_output מקבלת פרמטר אחד בשם wait, שהוא ערך בוליאני. כברירת מחדל, wait מוגדר כ-False. פרמטר זה קובע האם הפונקציה צריכה להמתין עד שיהיה פלט חדש להחליף את הפלט הקיים לפני שהיא מנקה אותו.

הפונקציה עצמה משמשת לניקוי הפלט של התא הנוכחי. ב-Jupyter Notebook או IPython, כאשר תא מייצר פלט, כמו טקסט מודפס או גרפים, הפלט הזה מוצג מתחת לתא. הפונקציה clear_output מאפשרת לנקות את הפלט הזה.

מימוש הפונקציה לא מסופק בקטע הקוד, כפי שמצוין על ידי שלוש הנקודות (...). שלוש הנקודות מייצגות מציין מקום לקוד האמיתי שמבצע את פעולת הניקוי של הפלט. ייתכן שמימוש הפונקציה כולל אינטראקציה עם ה-API של Jupyter Notebook או IPython כדי להסיר את הפלט הקיים מהתא.

בסך הכל, הפונקציה הזו מספקת דרך נוחה לנקות את הפלט של התא הנוכחי ב-Jupyter Notebook או IPython, מה שמקל על ניהול ועדכון הפלט המוצג במהלך סשנים אינטראקטיביים של קידוד.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

בצע המרת טקסט לדיבור (TTS) באמצעות שירות Edge TTS. בואו נעבור על יישומי הפונקציות הרלוונטיות אחת אחת:

1. `calculate_rate_string(input_value)`: פונקציה זו מקבלת ערך קלט ומחשבת את מחרוזת הקצב עבור הקול של TTS. ערך הקלט מייצג את מהירות הדיבור הרצויה, כאשר ערך של 1 מייצג מהירות רגילה. הפונקציה מחשבת את מחרוזת הקצב על ידי חיסור 1 מערך הקלט, הכפלתו ב-100, ולאחר מכן קובעת את הסימן בהתאם לשאלה האם ערך הקלט גדול או שווה ל-1. הפונקציה מחזירה את מחרוזת הקצב בפורמט "{sign}{rate}".

2. `make_chunks(input_text, language)`: פונקציה זו מקבלת טקסט קלט ושפה כפרמטרים. היא מחלקת את טקסט הקלט לקטעים על פי כללים ספציפיים לשפה. ביישום זה, אם השפה היא "אנגלית", הפונקציה מחלקת את הטקסט בכל נקודה (".") ומסירה רווחים מובילים או נגררים. לאחר מכן היא מוסיפה נקודה לכל קטע ומחזירה את רשימת הקטעים המסוננים.

3. `tts_file_name(text)`: פונקציה זו יוצרת שם קובץ עבור קובץ האודיו של TTS בהתבסס על טקסט הקלט. היא מבצעת מספר טרנספורמציות על הטקסט: הסרת נקודה נגררת (אם קיימת), המרת הטקסט לאותיות קטנות, הסרת רווחים מובילים ונגררים, והחלפת רווחים בקו תחתון. לאחר מכן היא מקצרת את הטקסט לאורך מקסימלי של 25 תווים (אם ארוך יותר) או משתמשת בטקסט המלא אם הוא ריק. לבסוף, היא יוצרת מחרוזת אקראית באמצעות מודול [`uuid`] ומשלבת אותה עם הטקסט המקוצר כדי ליצור את שם הקובץ בפורמט "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: פונקציה זו ממזגת מספר קבצי אודיו לקובץ אודיו אחד. היא מקבלת רשימה של נתיבי קבצי אודיו ונתיב פלט כפרמטרים. הפונקציה מאתחלת אובייקט ריק מסוג `AudioSegment` בשם [`merged_audio`]. לאחר מכן היא עוברת על כל נתיב קובץ אודיו, טוענת את קובץ האודיו באמצעות השיטה `AudioSegment.from_file()` מספריית `pydub`, ומוסיפה את קובץ האודיו הנוכחי לאובייקט [`merged_audio`]. לבסוף, היא מייצאת את האודיו הממוזג לנתיב הפלט שצוין בפורמט MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: פונקציה זו מבצעת את פעולת TTS באמצעות שירות Edge TTS. היא מקבלת רשימה של קטעי טקסט, מהירות הדיבור, שם הקול ונתיב שמירה כפרמטרים. אם מספר הקטעים גדול מ-1, הפונקציה יוצרת תיקייה לאחסון קבצי האודיו של הקטעים הבודדים. לאחר מכן היא עוברת על כל קטע, בונה פקודת Edge TTS באמצעות הפונקציה `calculate_rate_string()`, שם הקול וטקסט הקטע, ומבצעת את הפקודה באמצעות הפונקציה `os.system()`. אם ביצוע הפקודה מצליח, היא מוסיפה את נתיב קובץ האודיו שנוצר לרשימה. לאחר עיבוד כל הקטעים, היא ממזגת את קבצי האודיו הבודדים באמצעות הפונקציה `merge_audio_files()` ושומרת את האודיו הממוזג לנתיב השמירה שצוין. אם יש רק קטע אחד, היא יוצרת ישירות את פקודת Edge TTS ושומרת את האודיו לנתיב השמירה. לבסוף, היא מחזירה את נתיב השמירה של קובץ האודיו שנוצר.

6. `random_audio_name_generate()`: פונקציה זו יוצרת שם קובץ אודיו אקראי באמצעות מודול [`uuid`]. היא יוצרת UUID אקראי, ממירה אותו למחרוזת, לוקחת את שמונה התווים הראשונים, מוסיפה את הסיומת ".mp3", ומחזירה את שם קובץ האודיו האקראי.

7. `talk(input_text)`: פונקציה זו היא נקודת הכניסה הראשית לביצוע פעולת TTS. היא מקבלת טקסט קלט כפרמטר. היא קודם כל בודקת את אורך טקסט הקלט כדי לקבוע אם מדובר במשפט ארוך (600 תווים או יותר). בהתבסס על האורך ועל הערך של המשתנה `translate_text_flag`, היא קובעת את השפה ויוצרת את רשימת קטעי הטקסט באמצעות הפונקציה `make_chunks()`. לאחר מכן היא יוצרת נתיב שמירה עבור קובץ האודיו באמצעות הפונקציה `random_audio_name_generate()`. לבסוף, היא קוראת לפונקציה `edge_free_tts()` כדי לבצע את פעולת TTS ומחזירה את נתיב השמירה של קובץ האודיו שנוצר.

בסך הכל, פונקציות אלו פועלות יחד כדי לחלק את טקסט הקלט לקטעים, ליצור שם קובץ עבור קובץ האודיו, לבצע את פעולת TTS באמצעות שירות Edge TTS, ולמזג את קבצי האודיו הבודדים לקובץ אודיו אחד.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

היישום של שתי פונקציות: convert_to_text ו-run_text_prompt, וכן ההכרזה על שתי מחלקות: str ו-Audio.

הפונקציה convert_to_text מקבלת כקלט את audio_path ומבצעת תמלול של האודיו לטקסט באמצעות מודל בשם whisper_model. הפונקציה קודם כל בודקת אם הדגל gpu מוגדר כ-True. אם כן, נעשה שימוש ב-whisper_model עם פרמטרים מסוימים כמו word_timestamps=True, fp16=True, language='English', ו-task='translate'. אם הדגל gpu מוגדר כ-False, נעשה שימוש ב-whisper_model עם fp16=False. התמלול המתקבל נשמר בקובץ בשם 'scan.txt' ומוחזר כטקסט.

הפונקציה run_text_prompt מקבלת הודעה ו-chat_history כקלט. היא משתמשת בפונקציה phi_demo כדי ליצור תגובה מצ'אטבוט בהתבסס על ההודעה שהתקבלה. התגובה שנוצרה מועברת לפונקציה talk, שממירה את התגובה לקובץ אודיו ומחזירה את נתיב הקובץ. מחלקת Audio משמשת להצגת קובץ האודיו ולהשמעתו. האודיו מוצג באמצעות הפונקציה display מהמודול IPython.display, ואובייקט Audio נוצר עם הפרמטר autoplay=True, כך שהאודיו מתחיל להתנגן באופן אוטומטי. ה-chat_history מתעדכן עם ההודעה שהתקבלה והתגובה שנוצרה, ומוחזרים מחרוזת ריקה ו-chat_history המעודכן.

מחלקת str היא מחלקה מובנית בפייתון שמייצגת רצף של תווים. היא מספקת מגוון שיטות לעבודה עם מחרוזות, כמו capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, ועוד. שיטות אלו מאפשרות לבצע פעולות כמו חיפוש, החלפה, עיצוב ועיבוד של מחרוזות.

מחלקת Audio היא מחלקה מותאמת אישית שמייצגת אובייקט אודיו. היא משמשת ליצירת נגן אודיו בסביבת Jupyter Notebook. המחלקה מקבלת מגוון פרמטרים כמו data, filename, url, embed, rate, autoplay, ו-normalize. הפרמטר data יכול להיות מערך numpy, רשימת דגימות, מחרוזת שמייצגת שם קובץ או URL, או נתוני PCM גולמיים. הפרמטר filename משמש לציון קובץ מקומי לטעינת נתוני האודיו, והפרמטר url משמש לציון URL להורדת נתוני האודיו. הפרמטר embed קובע האם נתוני האודיו צריכים להיות מוטמעים באמצעות URI של נתונים או להיות מופנים למקור המקורי. הפרמטר rate מציין את קצב הדגימה של נתוני האודיו. הפרמטר autoplay קובע האם האודיו צריך להתחיל להתנגן באופן אוטומטי. הפרמטר normalize מציין האם נתוני האודיו צריכים להיות מנורמלים (מכוילים) לטווח האפשרי המרבי. מחלקת Audio מספקת גם שיטות כמו reload לטעינת נתוני האודיו מחדש מקובץ או URL, ותכונות כמו src_attr, autoplay_attr, ו-element_id_attr כדי לשלוף את התכונות המתאימות לאלמנט האודיו ב-HTML.

בסך הכל, הפונקציות והמחלקות הללו משמשות לתמלול אודיו לטקסט, יצירת תגובות אודיו מצ'אטבוט, והצגה והשמעה של אודיו בסביבת Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
